In [1]:
#fit notebook width
# from IPython.display import display, HTML

# display(
#     HTML(data="""
# <style>
#     div#notebook-container    { width: 95%; }
#     div#menubar-container     { width: 65%; }
#     div#maintoolbar-container { width: 99%; }
# </style>
# """))
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import time
from importlib import reload

""" *********************************************************************************************************************************** """

# MONGODB
from pymongo import MongoClient
from bson import  ObjectId
import bsonnumpy
#Ploting
import folium
from folium.plugins import FastMarkerCluster
from folium.plugins import MarkerCluster
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
# Data and processing
import datetime
import numpy as np
import pandas as pd
# Printing
import pprint
# parallel
import os
import ipyparallel as ipp
try :    
    rc = ipp.Client()
    parll = rc[:]
except OSError:
    print('ipyparallel not loaded')
""" *********************************************************************************************************************************** """
import sys
sys.path.append('git/PRIM/source/')
sys.path.append('git/PRIM/scripts/')

import Trips
import DataProcessing
import Plotting
import TripsProcessing
import OsmProcessing
import speed_matrix 
import CustomUtils
import osmMerger
from speed_matrix import SpeedMatrix
from affect_road_to_point_para import get_north_azimut
""" *********************************************************************************************************************************** """

%aimport pandas , numpy , matplotlib, seaborn, os, sys, ipyparallel, pprint, folium, datetime
sns.set()

Waiting for connection file: ~/.ipython/profile_default/security/ipcontroller-client.json
ipyparallel not loaded


In [6]:
client= MongoClient()
osmWays=client.osm.ways

In [8]:
segments = OsmProcessing.getSegments(osmWays)
segments = OsmProcessing.setOneWay(segments)


In [12]:
myPointsDF = DataProcessing.loadRawData(coyoteData)

In [14]:
myPointsDF = myPointsDF[myPointsDF.matching_road.isin(segments.index)]
myPointsDF.reset_index(drop=True, inplace=True)

In [ ]:
def discreteTimeID(pdf, nMinutes):
    if (nMinutes <= 0 or nMinutes > 60):
        print("Error : Number of minutes must be under 60")
        return
    return pdf.time.dt.hour * int(
        60 / nMinutes) + myPointsDF.time.dt.minute.apply(
            lambda x: int(x / nMinutes))


myPointsDF = myPointsDF.assign(heure=discreteTimeID(myPointsDF, 60))
myPointsDF = myPointsDF.assign(demi_heure=discreteTimeID(myPointsDF, 30))
myPointsDF = myPointsDF.assign(quart_heure=discreteTimeID(myPointsDF, 15))
myPointsDF = myPointsDF.assign(
    day=myPointsDF.time.apply(lambda x: pd.to_datetime(x).date()))

In [ ]:
myPointsDF.describe()

In [ ]:
roadGroups = myPointsDF.groupby("matching_road")

## Number of devices per segment

In [ ]:
plt.figure(figsize=(14, 4))
plt.subplot(121)
roadGroups.size().plot.box(
    showfliers=False, whis=3, label="NB devices per segment", showmeans=True)
plt.subplot(122)
roadGroups.size().plot.box(
    whis=3, label="NB devices per segment", showmeans=True)
cents = roadGroups.size().quantile(np.arange(0, 1.01, 0.1))
cents.index = ['min'] + ["C" + str(i) for i in range(10, 100, 10)] + ['max']
pd.DataFrame(cents).T

## Number of devices per (15/30/60) minutes (over all days)

In [ ]:
plt.figure(figsize=(18, 4))
plt.subplot(131)

quartGroups = myPointsDF.groupby(["matching_road", 'quart_heure'])
quartGroups.size().unstack('quart_heure').fillna(0).sum().plot.box(
    label='NB devices per 15 minutes', showmeans=True)

plt.subplot(132)
demiGroups = myPointsDF.groupby(["matching_road", 'demi_heure'])
demiGroups.size().unstack('demi_heure').fillna(0).sum().plot.box(
    label='NB devices per 30 minutes', showmeans=True)

plt.subplot(133)
heureGroups = myPointsDF.groupby(["matching_road", 'heure'])
heureGroups.size().unstack('heure').fillna(0).sum().plot.box(
    label='NB devices per 60 minutes', showmeans=True)

## points distribution over hours for each day

In [ ]:
myPointsDF.groupby(['day',
                    'heure']).size().unstack('heure').fillna(0).plot.bar(
                        color=plt.cm.hsv(np.arange(0, 1, 1 / 24)),
                        figsize=(24, 4))
plt.legend(ncol=12)

## points distribution over days for each hour

In [ ]:
myPointsDF.groupby(['day', 'heure']).size().unstack('day').fillna(0).plot.bar(
    color=plt.cm.hsv(np.arange(0, 1, 1 / 8)), figsize=(24, 8))
plt.legend(ncol=12)

## heatmaps  points distribution over quarters of hour for each day

In [ ]:
plt.figure(figsize=(18, 6))
plt.imshow(
    myPointsDF.groupby(['day', 'quart_heure']).size().unstack('day').fillna(0),
    aspect='auto')

## number of points per segment per  day per hour

In [ ]:
heureGroups = myPointsDF.groupby(["matching_road", 'heure'])
heureGroups.size(
).unstack('heure').fillna(0).mean(level='matching_road').plot.box(
    figsize=(24, 6),
    showfliers=False,
    whis=3,
    title="average NB devices per segment per hour per day(outliers omitted)",
    showmeans=True)
heureGroups = myPointsDF.groupby(['day', "matching_road", 'heure'])
heureGroups.size().unstack('heure').fillna(0).mean(
    level='matching_road').plot.box(
        figsize=(24, 6),
        showfliers=True,
        whis=3,
        title="average NB devices per segment per hour per day",
        showmeans=True)

cents = heureGroups.size().unstack('heure').fillna(0).mean(
    level='matching_road').quantile(np.arange(0, 1.01, 0.1))
cents.index = ['min'] + ["C" + str(i) for i in range(10, 100, 10)] + ['max']
pd.DataFrame(cents).T

## Number of points per segment for each hour over all period

In [ ]:
plt.figure(figsize=(20, 6))
segmentsDenseDF = myPointsDF.groupby(
    ['day', "matching_road",
     'quart_heure']).size().unstack(['day']).unstack('quart_heure').fillna(0)
plt.imshow(np.log(segmentsDenseDF), aspect='auto')
segmentsDenseDF

In [ ]:
date = segmentsDenseDF.columns.levels[0][0]
hour = 91
maxDens = segmentsDenseDF[date][hour].max()

folium_map = folium.Map(
    location=[48.14301, -1.69537], zoom_start=13, tiles="OpenStreetMap")

[
    folium.PolyLine(
        locations=[[lo[1], lo[0]] for lo in segments[segments.index == segID]
                   ['loc'].iloc[0]['coordinates']],
        color=matplotlib.colors.rgb2hex(
            plt.cm.hot_r(
                np.log(segmentsDenseDF.loc[segID].loc[date].loc[hour] + 1) /
                (np.log(maxDens) + 1)))).add_to(folium_map)
    for segID in segmentsDenseDF.index.values
]
folium_map.save('segmentsDensity.html')

In [ ]:
segmentsSpeedDF = myPointsDF.groupby(
    ['day', "matching_road", 'quart_heure']).mean()['speed'].unstack(
        ['day']).unstack('quart_heure').fillna(0)

In [ ]:
date = datetime.date(2018, 11, 18)
hour = 91
maxDens = segmentsSpeedDF[date][hour].max()

folium_map = folium.Map(
    location=[48.14301, -1.69537], zoom_start=13, tiles="OpenStreetMap")

[
    folium.PolyLine(
        locations=[[lo[1], lo[0]] for lo in segments[segments.index == segID]
                   ['loc'].iloc[0]['coordinates']],
        color=matplotlib.colors.rgb2hex(
            plt.cm.hot_r(segmentsSpeedDF.loc[segID].loc[date].loc[hour] /
                         maxDens))).add_to(folium_map)
    for segID in segmentsSpeedDF.index.values
]

folium_map.save('segmentsSpeed.html')

In [ ]:
segmentsDenseDF.head(1)

In [ ]:
segmentsDenseDF[[datetime.date(2018, 11, x) for x in range(19, 23)]]

In [ ]:
segmentsDenseDF[[datetime.date(2018, 11, x)
                 for x in range(19, 23)]].to_csv("segments density.csv")
segmentsSpeedDF[[datetime.date(2018, 11, x)
                 for x in range(19, 23)]].to_csv("segments speed.csv")

In [ ]:
segmentsDenseDF[[datetime.date(2018, 11, x)
                 for x in range(16, 17)]].to_csv("segments density TEST.csv")
segmentsSpeedDF[[datetime.date(2018, 11, x)
                 for x in range(16, 17)]].to_csv("segments speed TEST.csv")

In [ ]:
folium_map = folium.Map(
    location=[48.14301, -1.69537], zoom_start=13, tiles="OpenStreetMap")

[
    folium.PolyLine(
        locations=[[lo[1], lo[0]]
                   for lo in hierarchy['loc'].iloc[idx]['coordinates']],
        popup=str(hierarchy.length.iloc[idx]),
        color=matplotlib.colors.rgb2hex(
            plt.cm.hot_r(hierarchy.length.iloc[idx] /
                         hierarchy.length.max()))).add_to(folium_map)
    for idx in range(len(hierarchy))
]

folium_map.save('distances.html')

In [ ]:
hierarchy.to_pickle("hierarchy.pckl")

# Speed Matrix

In [10]:
def setDateAndTimeIndex(df):
    idx=[pd.to_datetime(df.columns.values).date,pd.to_datetime(df.columns.values).time]
    mIdx=pd.MultiIndex.from_arrays(idx,names=['day','time'])
    df.set_axis(mIdx,axis=1,inplace=True)
    

def getMaxSpeed(speedMX,segmentsMaxSpeed):
    observedMax= speedMX.max(axis=1)
    maxSpeed= pd.Series(segmentsMaxSpeed[np.array([*map(lambda x : int(x[:-2]),speedMX.index.values)])].values,index=speedMX.index)
    return maxSpeed.fillna(observedMax)

def setWeightedSpeed(speedMX,densityMX,meanMX,densityThreshold):
    # weighted mean
    speedMX = ((densityMX>=densityThreshold)*speedMX).add(
                  (densityMX<densityThreshold)*(((densityMX/densityThreshold)*speedMX).add( ((densityThreshold-densityMX)/densityThreshold)*meanMX , fill_value = 0) )
                ,fill_value=0)
    return speedMX

def setMaxSpeed(speedMX,maxSpeedMX):
    speedMX.apply(lambda x : x.fillna(maxSpeedMX,inplace=True))
    

In [ ]:
# densityMX = pd.DataFrame(data=np.ones(speedMX.shape),columns=speedMX.columns,index=speedMX.index)

# meanMX=speedMX.groupby(by=speedMX.columns.get_level_values(1),axis=1).mean()

# maxSpeedMX = getMaxSpeed(speedMX,segmentsMeta.maxSpeed)

# speedMX = setWeightedSpeed(speedMX,densityMX,meanMX,2)

# setMaxSpeed(speedMX,maxSpeedMX)

In [ ]:
def getColumnSlice(columns,minTime=datetime.time(17),maxTime=datetime.time(20)):
    return columns[np.fromiter(map(lambda x : (x.time()>minTime and x.time()<maxTime),columns),np.bool)]
    

In [14]:
osmMerger = reload(osmMerger)

In [11]:
from mongoConnection import *

In [10]:
sm = SpeedMatrix("congestion", "coyote2", "osm", "roads")
roads_ids = segments.index.values.tolist()
speeds,counts = sm.get_speed_matrix(15, roads_ids, 14, 19)

In [ ]:
reload()

In [14]:
speeds.to_pickle("speeds1419.pckl")
counts.to_pickle("counts1419.pckl")

In [19]:
speeds.isna().sum(axis=1).describe()

count    6471.000000
mean      230.956266
std        89.311379
min         0.000000
25%       184.000000
50%       264.000000
75%       303.000000
max       319.000000
dtype: float64

In [15]:
mergeResults,fmap = osmMerger.mergeSegments(minValidData = osmMerger.MIN_VALID_DATA, weights=osmMerger.WEIGHTS,speedsMx=speeds)

Computing raw data : 
first time call or reset : starting counter
getting segments and meta :


KeyboardInterrupt: 

In [651]:
snapShot=mergedSegments[(mergedSegments.nonNullProp>0.8)]

In [652]:
mergeResults = inversedIndex.loc[inversedIndex.isin(snapShot.index)]

In [29]:
fmap  = Plotting.getFoliumMap()
layer = getMergeLayer(fmap,mergeResults,snapShot,segmentsMeta,'layer ahahah')
fmap  = Plotting.stackHistotyLayers([layer],fmap)
fmap.save('Roads.html')

In [28]:
fmap  = Plotting.getFoliumMap()
layer = Plotting.saveBigMergesMap(mergeResults,segmentsMeta,fmap=fmap,name="biggest 15")
fmap  = Plotting.stackHistotyLayers([layer],fmap)
fmap.save('steps.html')

In [656]:
fmap  = Plotting.getFoliumMap()
layers = makePercentageSnapShots(fmap,mergedSegments,inversedIndex,segmentsMeta)
fmap  = Plotting.stackHistotyLayers(layers,fmap)
fmap.save('RoadsPer10Percents.html')